In [1]:
from dotenv import load_dotenv

load_dotenv()
import os
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [2]:
import bs4

1. Data Loading

In [3]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader(["https://my.clevelandclinic.org/health/symptoms/10880-fever","https://en.wikipedia.org/wiki/Fever"])
data = loader.load()
data

[Document(metadata={'source': 'https://my.clevelandclinic.org/health/symptoms/10880-fever', 'title': 'Fever Symptoms & Causes: What Is Considered a Fever?', 'description': 'A fever is when your body temperature rises higher than normal. A fever itself isn’t an illness. Rather, it’s a symptom for a wide range of health conditions.', 'language': 'en'}, page_content='Fever Symptoms & Causes: What Is Considered a Fever?Locations:Abu Dhabi|Canada|Florida|London|Nevada|Ohio|800.223.2273100 Years of Cleveland ClinicMyChartNeed Help?GivingCareersSearchClevelandClinic.orgFind A DoctorLocations & DirectionsPatients & VisitorsHealth LibraryInstitutes & DepartmentsAppointmentsHome/Health Library/Symptoms/FeverAdvertisementAdvertisementFeverA fever is when your body temperature is higher than your normal average temp. Most providers say a fever is either 100.0 F (37.8 C) or 100.4 F (38 C). When you have a fever, it’s your body’s normal response to fighting an infection or illness. Fevers usually ar

In [5]:
data[0].page_content

'Fever Symptoms & Causes: What Is Considered a Fever?Locations:Abu Dhabi|Canada|Florida|London|Nevada|Ohio|800.223.2273100 Years of Cleveland ClinicMyChartNeed Help?GivingCareersSearchClevelandClinic.orgFind A DoctorLocations & DirectionsPatients & VisitorsHealth LibraryInstitutes & DepartmentsAppointmentsHome/Health Library/Symptoms/FeverAdvertisementAdvertisementFeverA fever is when your body temperature is higher than your normal average temp. Most providers say a fever is either 100.0 F (37.8 C) or 100.4 F (38 C). When you have a fever, it’s your body’s normal response to fighting an infection or illness. Fevers usually aren’t a serious concern. They typically go away when the infection passes.AdvertisementCleveland Clinic is a non-profit academic medical center. Advertising on our site helps support our mission. We do not endorse non-Cleveland Clinic products or services. PolicyCare at Cleveland ClinicFind a Primary Care ProviderSchedule an AppointmentContentsArrow DownOverviewPos

In [6]:
# We need to split the text using Character Text Split such that it sshould not increse token size
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap  = 200,
)
texts = text_splitter.split_documents(data)


In [7]:
texts

[Document(metadata={'source': 'https://my.clevelandclinic.org/health/symptoms/10880-fever', 'title': 'Fever Symptoms & Causes: What Is Considered a Fever?', 'description': 'A fever is when your body temperature rises higher than normal. A fever itself isn’t an illness. Rather, it’s a symptom for a wide range of health conditions.', 'language': 'en'}, page_content='Fever Symptoms & Causes: What Is Considered a Fever?Locations:Abu Dhabi|Canada|Florida|London|Nevada|Ohio|800.223.2273100 Years of Cleveland ClinicMyChartNeed Help?GivingCareersSearchClevelandClinic.orgFind A DoctorLocations & DirectionsPatients & VisitorsHealth LibraryInstitutes & DepartmentsAppointmentsHome/Health Library/Symptoms/FeverAdvertisementAdvertisementFeverA fever is when your body temperature is higher than your normal average temp. Most providers say a fever is either 100.0 F (37.8 C) or 100.4 F (38 C). When you have a fever, it’s your body’s normal response to fighting an infection or illness. Fevers usually ar

In [8]:
len(texts)

134

2. Create Embeddings

In [9]:
from langchain_openai import OpenAIEmbeddings

In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [11]:
from langchain_community.vectorstores import FAISS

In [12]:
db = FAISS.from_documents(texts, embeddings)

In [13]:
query = "What are the symptoms of fever?"
similar_docs = db.similarity_search(query,3)

In [14]:
similar_docs

[Document(id='6298e033-a7d2-4e2a-bb26-7a47726d9a8b', metadata={'source': 'https://en.wikipedia.org/wiki/Fever', 'title': 'Fever - Wikipedia', 'language': 'en'}, page_content='Associated symptoms[edit]\nA fever is usually accompanied by sickness behavior, which consists of lethargy, depression, loss of appetite, sleepiness, hyperalgesia, dehydration,[21][22] and the inability to concentrate. Sleeping with a fever can often cause intense or confusing nightmares, commonly called "fever dreams".[23] Mild to severe delirium (which can also cause hallucinations) may also present itself during high fevers.[24]\n\nDiagnosis[edit]\n\nTemperature classification\n Core (rectal, esophageal, etc.) Hypothermia <35.0\xa0°C (95.0\xa0°F)[25] Normal 36.5–37.5\xa0°C (97.7–99.5\xa0°F)[26][27] Fever >37.5 or 38.3\xa0°C (99.5 or 100.9\xa0°F)[1][8] Hyperthermia >37.5 or 38.3\xa0°C (99.5 or 100.9\xa0°F)[1][8] Hyperpyrexia >40.0 or 41.0\xa0°C (104.0 or 105.8\xa0°F)[28][29]'),
 Document(id='c27116b2-2643-4f63-a

In [14]:
similar_docs[0].page_content


'Associated symptoms[edit]\nA fever is usually accompanied by sickness behavior, which consists of lethargy, depression, loss of appetite, sleepiness, hyperalgesia, dehydration,[21][22] and the inability to concentrate. Sleeping with a fever can often cause intense or confusing nightmares, commonly called "fever dreams".[23] Mild to severe delirium (which can also cause hallucinations) may also present itself during high fevers.[24]\n\nDiagnosis[edit]\n\nTemperature classification\n Core (rectal, esophageal, etc.) Hypothermia <35.0\xa0°C (95.0\xa0°F)[25] Normal 36.5–37.5\xa0°C (97.7–99.5\xa0°F)[26][27] Fever >37.5 or 38.3\xa0°C (99.5 or 100.9\xa0°F)[1][8] Hyperthermia >37.5 or 38.3\xa0°C (99.5 or 100.9\xa0°F)[1][8] Hyperpyrexia >40.0 or 41.0\xa0°C (104.0 or 105.8\xa0°F)[28][29]'

In [15]:
similar_docs[1].page_content

"Medical conditionFeverOther namesPyrexia, febrile response, febrile[1]Person with feverSpecialtyInfectious disease, pediatricsSymptomsInitially: shivering, feeling cold, chills[2]Later: flushed, sweating[3]ComplicationsFebrile seizure[4]CausesVirus, bacteria, increase in the body's temperature set point[5][6]Diagnostic methodTemperature higher than the normal range of 37.2 and 38.3\xa0°C (99.0 and 100.9\xa0°F)[1][7][8]Differential diagnosisHyperthermia[1]TreatmentBased on underlying cause, not required for fever itself[2][9]MedicationIbuprofen, paracetamol (acetaminophen)[9][10]FrequencyCommon[2][11]"

3. LLM Call

In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [17]:

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model_name="gpt-4",
)
chain = load_qa_chain(llm=llm, chain_type="stuff",verbose=True)



C:\Users\Mohammed Ammar\AppData\Local\Temp\ipykernel_21800\4204476506.py:5: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm=llm, chain_type="stuff",verbose=True)


In [18]:
query = "what is fever?"
chain.run(input_documents=similar_docs, question=query)

C:\Users\Mohammed Ammar\AppData\Local\Temp\ipykernel_21800\272985106.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=similar_docs, question=query)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Associated symptoms[edit]
A fever is usually accompanied by sickness behavior, which consists of lethargy, depression, loss of appetite, sleepiness, hyperalgesia, dehydration,[21][22] and the inability to concentrate. Sleeping with a fever can often cause intense or confusing nightmares, commonly called "fever dreams".[23] Mild to severe delirium (which can also cause hallucinations) may also present itself during high fevers.[24]

Diagnosis[edit]

Temperature classification
 Core (rectal, esophageal, etc.) Hypothermia <35.0 °C (95.0 °F)[25] Normal 36.5–37.5 °C (97.7–99.5 °F)[26][27] Fever >37.5 or 38.3 °C (99.5 or 100.9 °F)[1][8] Hyperthermia >37.5 or 38.3 °C (99.5 or 100.9 °F)[1][8] Hyperpyrexia >4

"A fever, also known as Pyrexia or febrile response, is a medical condition usually characterized by an increase in the body's temperature set point, causing the temperature to be higher than the normal range of 37.2 and 38.3 °C (99.0 and 100.9 °F). Symptoms of fever may initially include shivering, feeling cold, and chills, which later transition to feeling flushed and sweating. Other symptoms can include body aches, headaches, fatigue, and possibly faster heart beats. Fever is a common response to infections, being caused by factors such as viruses or bacteria. Treatment is usually based on addressing the underlying cause rather than the fever itself."

In [19]:
query = "what are the symptoms of fever?"
chain.run(input_documents=similar_docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Associated symptoms[edit]
A fever is usually accompanied by sickness behavior, which consists of lethargy, depression, loss of appetite, sleepiness, hyperalgesia, dehydration,[21][22] and the inability to concentrate. Sleeping with a fever can often cause intense or confusing nightmares, commonly called "fever dreams".[23] Mild to severe delirium (which can also cause hallucinations) may also present itself during high fevers.[24]

Diagnosis[edit]

Temperature classification
 Core (rectal, esophageal, etc.) Hypothermia <35.0 °C (95.0 °F)[25] Normal 36.5–37.5 °C (97.7–99.5 °F)[26][27] Fever >37.5 or 38.3 °C (99.5 or 100.9 °F)[1][8] Hyperthermia >37.5 or 38.3 °C (99.5 or 100.9 °F)[1][8] Hyperpyrexia >4

'The symptoms of fever can include:\n\n- Initially: shivering, feeling cold, chills\n- Later: feeling flushed, sweating\n- Lethargy\n- Depression\n- Loss of appetite\n- Sleepiness \n- Hyperalgesia (increased sensitivity to pain)\n- Dehydration\n- Inability to concentrate\n- Confusing nightmares (also known as "fever dreams")\n- Possible mild to severe delirium, which can cause hallucinations\n\nAdditional symptoms in children may include:\n\n- Lack of appetite\n- Earache or pulling at their ears\n- High-pitched crying\n- Fussiness\n- Paleness or flushness\n- Excessive thirst\n- Decreased urination\n'

In [20]:
query = "what are things present in bank"
chain.run(input_documents=similar_docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Associated symptoms[edit]
A fever is usually accompanied by sickness behavior, which consists of lethargy, depression, loss of appetite, sleepiness, hyperalgesia, dehydration,[21][22] and the inability to concentrate. Sleeping with a fever can often cause intense or confusing nightmares, commonly called "fever dreams".[23] Mild to severe delirium (which can also cause hallucinations) may also present itself during high fevers.[24]

Diagnosis[edit]

Temperature classification
 Core (rectal, esophageal, etc.) Hypothermia <35.0 °C (95.0 °F)[25] Normal 36.5–37.5 °C (97.7–99.5 °F)[26][27] Fever >37.5 or 38.3 °C (99.5 or 100.9 °F)[1][8] Hyperthermia >37.5 or 38.3 °C (99.5 or 100.9 °F)[1][8] Hyperpyrexia >4

"I'm sorry, but the provided context doesn't include information about what is present in a bank."